### Import thư viện

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

### Đọc dữ liệu từ file csv

In [ ]:
df = pd.read_csv("./SampleSuperstore.csv",encoding = "ISO-8859-1")

## Tiền xử lý

**In 5 dòng đầu tiên của dữ liệu**

In [ ]:
df.head()

**Thông tin số dòng và số cột**

In [ ]:
df.shape

**Mô tả dữ liệu**

In [ ]:
df.describe()

**Thông tin cột**

In [ ]:
df.columns

**Kiểu dữ liệu của từng cột**

In [ ]:
df.dtypes

**Số giá trị null của từng cột**

In [ ]:
df.isnull().sum()

**Loại bỏ các dòng có xuất hiện it nhất 1 giá trị null trong dataframe**

Do dữ liệu có hơn 20000 dòng nên việc loại bỏ các giá trị null rơi vào khoảng 2000 dòng không ảnh hưởng nhiều

In [ ]:
df.dropna(inplace=True)

Kiểm tra lại dataframe

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

**Loại bỏ cột không cần thiết**

Các cột bị loại bỏ:
- Cột `discount_price__currency` và `price_detail__currency` đều chỉ có 1 giá trị duy nhất là `INR`
- Cột `is_wishlisted` có 1 giá trị duy nhất là `False`
- Cột `discount_price__price_string` có dữ liệu kiểu string và được tổng hợp từ 2 cột `discount_price__currency` và `discount_price__amount` nên ta sẽ loại bỏ cột này. Tương tự với cột `price_detail__price_string`
- Cột `id`, `url` không có thông tin hữu ích cho việc trực quan hóa và mô hình hóa
- Cột `avg_rating_recent` có giá trị dữ liệu từng dòng bằng với cột `rating` nên ta chọn loại bỏ cột `avg_rating_recent` để tránh dư thừa cột 

loai bo cot country vi chi co 1 gia tri duy nhat la United Stated

In [ ]:
a = df.groupby(['Country'])['Country'].count()
a

In [ ]:
print(df["discount_price__currency"].unique())
print(df["price_detail__currency"].unique())
print(df['is_wishlisted'].unique())

In [ ]:
print(df['avg_rating_recent'].equals(df['rating']))

In [ ]:
unused_columns = [
    'discount_price__currency', 
    'price_detail__currency',
    'discount_price__price_string',
    'price_detail__price_string',
    'is_wishlisted',
    'url',
    'id',
    'avg_rating_recent']

df.drop(columns=unused_columns, inplace=True)
df.columns

**Chuyển về kiểu dữ liệu datetime**

Áp dung cho 2 cột `created` và `published_time`

In [ ]:
df["created"] = pd.to_datetime(df["created"])
df["published_time"] = pd.to_datetime(df["published_time"])

Kiểu tra kiểu dữ liệu

In [ ]:
df.dtypes

Cách trích xuất thông tin từ dữ liệu datetime

In [ ]:
df['published_time'].dt.hour

**Chuyển giá trị tiền tệ từ INR thành USD**

USD là đơn vị tiền tệ phổ biến nên chuyển từ INR sang USD để có thể hiểu rõ hơn. Tại thời điểm thực hiện, 1 INR bằng khoảng 0.012 USD

In [ ]:
df['discount_price__amount'] = df['discount_price__amount'].apply(lambda x: x * 0.012)
df['price_detail__amount'] = df['price_detail__amount'].apply(lambda x: x * 0.012)

In [ ]:
df.head()

## Đưa ra câu hỏi và câu trả lời

**Câu 4:** Hãy phân tích về các loại sản phẩm được bán ở siêu thị này dựa trên các yếu tố: sales, profit, quantity,...

**Câu 5:** Hãy phân tích về các sự kiện discount của cửa hàng và mối liên hệ giữa Discount và các yếu tố khác.

### Câu 4: Hãy phân tích về các loại sản phẩm được bán ở siêu thị này dựa trên các yếu tố: sales, profit, quantity,...

Để trả lời câu hỏi này, ta sẽ chia ra trả lời 3 câu hỏi nhỏ, dựa trên 3 yếu tố là: Sales, Profit, Quantity.

#### 1. Sales

Tính tổng số lượng các sản phẩm đã bán dựa theo Category và Sub-category.

In [37]:
# import make_subplots function to create subplot
# Thêm make_subplots để tạo 
from plotly.subplots import make_subplots

import plotly.graph_objects as go
## CREATE NEW TABLE
# group by category
sales_per_category = df[['Category', 'Sales']].groupby('Category').sum().reset_index()

# sort values by Sales
sales_per_category.sort_values(by='Sales', ascending=False, inplace=True)

## PLOT
fig = px.bar(sales_per_category, x='Category', y='Sales')
fig.update_layout(title={'text': "Sales by Category",
                        'y':0.95,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                  xaxis_title='Category',
                  yaxis_title='Sales',
                  yaxis_tickprefix='$',
                  yaxis_tickformat=',.2f')
fig.show()
sales_per_category

,Category,Sales
2,Technology,836154.0330
0,Furniture,741999.7953
1,Office Supplies,719047.0320
